In [ ]:
import numpy as np
import pandas as pd
import os
import gzip
import io
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Предобработка логов

In [ ]:
# vibo: папка где лежат сырые логи
# vibo: ВАЖНО!!! ФАЙЛЫ В ПАПКЕ ДОЛЖНЫ БЫТЬ ОТСОРТИРОВАНЫ ОТ СТАРЫХ К НОВЫМ
# (latest.log...ok - должен прочитаться последним)
link = '/content/drive/MyDrive/minecraft/logs/'

In [ ]:
# vibo: задаем имена установленных плагинов
plug_names = ['WorldEdit', 'WorldGuard', 'HolographicDisplays']

In [ ]:
# vibo: лист для имен файлов в папке
file_names = []
# vibo: лист для логов
row_logs = []

for root, dirs, files in os.walk(link):  
    for filename in files:
      file_names.append(filename)

file_names = sorted(file_names)

for filename in file_names:
  # vibo: в папке файлы двух видов *log.gz и *.log (один последний)
  # vibo: обрабатываем файлы *log.gz
  if filename.endswith("gz"):
    # vibo: создаем путь до файла
    path = link + filename
    # vibo: разархивируем и читаем с учетом кодировки
    with gzip.open(path, 'rb') as f:
      with io.TextIOWrapper(f, encoding='utf-8') as decoder:
        # vibo: в content - полное содержимое одного лога
        content = decoder.read()
        # vibo: обрезаем перенос строк
        for i in content.split('\n'):
          # vibo: выделяем из имени файла дату и добавляем ее в начало строки
          row_logs.append(filename[:-9] + ' ' + i)
          # vibo: сохраняем крайнюю дату она понадобится ниже для последнего файла без даты
          last_date = filename[:-9]
    print(f'{filename}...ok' )

  # vibo: это обработка файла latest.log
  else:
    with open(link + 'latest.log', encoding="UTF-8") as file_in: 
        for line in file_in:
            messege = line.rstrip("\n")
            row_logs.append(last_date + ' ' + messege)
    print(f'{filename}...ok' )

2023-02-13-1.log.gz...ok
2023-02-13-2.log.gz...ok
2023-02-13-3.log.gz...ok
2023-02-13-4.log.gz...ok
2023-02-14-1.log.gz...ok
2023-02-14-2.log.gz...ok
2023-02-14-3.log.gz...ok
2023-02-17-1.log.gz...ok
2023-03-03-1.log.gz...ok
2023-03-03-2.log.gz...ok
2023-03-03-3.log.gz...ok
2023-03-04-1.log.gz...ok
2023-03-04-2.log.gz...ok
2023-03-04-3.log.gz...ok
2023-03-04-4.log.gz...ok
2023-03-04-5.log.gz...ok
2023-03-05-1.log.gz...ok
2023-03-05-2.log.gz...ok
2023-03-05-3.log.gz...ok
2023-03-06-1.log.gz...ok
2023-03-06-2.log.gz...ok
2023-03-06-3.log.gz...ok
2023-03-06-4.log.gz...ok
2023-03-07-1.log.gz...ok
2023-03-08-1.log.gz...ok
2023-03-08-2.log.gz...ok
2023-03-08-3.log.gz...ok
2023-03-09-1.log.gz...ok
latest.log...ok


In [ ]:
# vibo: проверяем последнюю строку
row_logs[3304]

'2023-03-05 [14:54:55] [Server thread/INFO]: vibo fell out of the world'

In [ ]:
# vibo: делаем датафрейм
row_logs = pd.DataFrame({
    'row_logs': row_logs
})
row_logs

,row_logs
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...
4,2023-02-13
...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...


In [ ]:
# vibo: выделяем дату
row_logs['date'] = row_logs['row_logs'].map(lambda x: x[:11])
# vibo: сохраняем в отдельный столбец оставшиеся данные
row_logs['row_logs_split_0'] = row_logs['row_logs'].map(lambda x: x[11:])
row_logs

,row_logs,date,row_logs_split_0
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...
4,2023-02-13,2023-02-13,
...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...


In [ ]:
# vibo: проверяем длину каждой строки, где-то остались пустые строки (len = 0)
row_logs['__len__'] = row_logs['row_logs_split_0'].map(lambda x: len(x))
row_logs

,row_logs,date,row_logs_split_0,__len__
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,60
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,85
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,53
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,117
4,2023-02-13,2023-02-13,,0
...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,102
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,55
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,165
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,71


In [ ]:
row_logs[row_logs['__len__'] == 0]

,row_logs,date,row_logs_split_0,__len__
4,2023-02-13,2023-02-13,,0
234,2023-02-13,2023-02-13,,0
405,2023-02-13,2023-02-13,,0
571,2023-02-13,2023-02-13,,0
574,2023-02-14,2023-02-14,,0
752,2023-02-14,2023-02-14,,0
934,2023-02-14,2023-02-14,,0
1217,2023-02-17,2023-02-17,,0
1406,2023-03-03,2023-03-03,,0
1656,2023-03-03,2023-03-03,,0


In [ ]:
# vibo: удалим 23 строки
row_logs[row_logs['__len__'] == 0].count()

row_logs            28
date                28
row_logs_split_0    28
__len__             28
dtype: int64

In [ ]:
# vibo: удаляем 0, получили 5083 информативных сообщения
row_logs = row_logs[row_logs['__len__'] != 0]
row_logs

,row_logs,date,row_logs_split_0,__len__
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,60
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,85
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,53
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,117
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,60
...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,102
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,55
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,165
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,71


In [ ]:
# vibo: выделяем время
row_logs['time'] = row_logs['row_logs_split_0'].map(lambda x: x.split('] [')[0][1:])
# vibo: проверяем длину строки в столбце времени
row_logs['__len__'] = row_logs['time'].map(lambda x: len(x))
row_logs

<ipython-input-18-69937498586c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_logs['time'] = row_logs['row_logs_split_0'].map(lambda x: x.split('] [')[0][1:])
<ipython-input-18-69937498586c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_logs['__len__'] = row_logs['time'].map(lambda x: len(x))


,row_logs,date,row_logs_split_0,__len__,time
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,8,21:31:47
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,8,21:31:48
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,8,21:31:48
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,8,21:31:48
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,8,21:34:03
...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,8,18:50:50
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,8,18:50:50
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,8,18:50:50
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,8,18:52:31


In [ ]:
# vibo: находим нестандартный лог (Reason: 1с) без времени, 10 значений
row_logs[row_logs['__len__'] != 8]

,row_logs,date,row_logs_split_0,__len__,time
1600,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1602,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1605,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1607,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1610,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1612,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1615,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1617,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1626,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с
1628,2023-03-03 Reason: 1с,2023-03-03,Reason: 1с,9,eason: 1с


In [ ]:
# vibo: удалим сообщения без времени
row_logs = row_logs[row_logs['__len__'] == 8]

In [ ]:
# vibo: итого осталось сообщений
row_logs

,row_logs,date,row_logs_split_0,__len__,time
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,8,21:31:47
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,8,21:31:48
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,8,21:31:48
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,8,21:31:48
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,8,21:34:03
...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,8,18:50:50
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,8,18:50:50
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,8,18:50:50
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,8,18:52:31


In [ ]:
# vibo: сохраняем часть лога без времени в отдельный столбце
row_logs['row_logs_split_1'] = row_logs['row_logs_split_0'].map(lambda x: x.split('] [')[1])
# vibo: удаляем промежуточный столбец
row_logs = row_logs.drop(['__len__'], axis=1)

row_logs

<ipython-input-22-d28dce669a36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_logs['row_logs_split_1'] = row_logs['row_logs_split_0'].map(lambda x: x.split('] [')[1])


,row_logs,date,row_logs_split_0,time,row_logs_split_1
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,21:31:47,ServerMain/INFO]: Building unoptimized datafixer
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,21:31:48,ServerMain/ERROR]: Failed to load properties f...
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,21:31:48,ServerMain/WARN]: Failed to load eula.txt
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,21:31:48,ServerMain/INFO]: You need to agree to the EUL...
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,21:34:03,ServerMain/INFO]: Building unoptimized datafixer
...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,18:50:50,User Authenticator #5/INFO]: UUID of player AR...
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,18:50:50,Server thread/INFO]: ARTEXX joined the game
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,18:50:50,Server thread/INFO]: ARTEXX[/[0:0:0:0:0:0:0:1]...
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,18:52:31,Server thread/INFO]: ARTEXX issued server comm...


In [ ]:
# vibo: выделяем стандартные сообщения сервера, остальныю часть сохраняем в отдельный столбец
row_logs['serv_teg'] = row_logs['row_logs_split_1'].map(lambda x: x.split(']:')[0])
row_logs['row_logs_split_2'] = row_logs['row_logs_split_1'].map(lambda x: x.split(']:')[1])

row_logs

,row_logs,date,row_logs_split_0,time,row_logs_split_1,serv_teg,row_logs_split_2
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,21:31:47,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,21:31:48,ServerMain/ERROR]: Failed to load properties f...,ServerMain/ERROR,Failed to load properties from file: server.p...
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,21:31:48,ServerMain/WARN]: Failed to load eula.txt,ServerMain/WARN,Failed to load eula.txt
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,21:31:48,ServerMain/INFO]: You need to agree to the EUL...,ServerMain/INFO,You need to agree to the EULA in order to run...
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,21:34:03,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer
...,...,...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,18:50:50,User Authenticator #5/INFO]: UUID of player AR...,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,18:50:50,Server thread/INFO]: ARTEXX joined the game,Server thread/INFO,ARTEXX joined the game
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,18:50:50,Server thread/INFO]: ARTEXX[/[0:0:0:0:0:0:0:1]...,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,18:52:31,Server thread/INFO]: ARTEXX issued server comm...,Server thread/INFO,ARTEXX issued server command: /farwand


In [ ]:
# vibo: заполняем имена плагинов для сообщений
row_logs['plug_name'] = row_logs['row_logs_split_2'].map(lambda x: [y for y in plug_names if y in x])
# vibo: делаем вспомогательный столбец, если плагин есть 1, нет 0
row_logs['__plug__'] = row_logs['plug_name'].map(lambda x: 1 if len(x) == 1 else 0)

row_logs

,row_logs,date,row_logs_split_0,time,row_logs_split_1,serv_teg,row_logs_split_2,plug_name,__plug__
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,21:31:47,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer,[],0
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,21:31:48,ServerMain/ERROR]: Failed to load properties f...,ServerMain/ERROR,Failed to load properties from file: server.p...,[],0
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,21:31:48,ServerMain/WARN]: Failed to load eula.txt,ServerMain/WARN,Failed to load eula.txt,[],0
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,21:31:48,ServerMain/INFO]: You need to agree to the EUL...,ServerMain/INFO,You need to agree to the EULA in order to run...,[],0
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,21:34:03,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer,[],0
...,...,...,...,...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,18:50:50,User Authenticator #5/INFO]: UUID of player AR...,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,[],0
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,18:50:50,Server thread/INFO]: ARTEXX joined the game,Server thread/INFO,ARTEXX joined the game,[],0
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,18:50:50,Server thread/INFO]: ARTEXX[/[0:0:0:0:0:0:0:1]...,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,[],0
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,18:52:31,Server thread/INFO]: ARTEXX issued server comm...,Server thread/INFO,ARTEXX issued server command: /farwand,[],0


In [ ]:
# vibo: заменил [] на NaN
row_logs['plug_name'] = row_logs[row_logs['__plug__'] == 1]['plug_name'].map(lambda x: x[0])

row_logs

,row_logs,date,row_logs_split_0,time,row_logs_split_1,serv_teg,row_logs_split_2,plug_name,__plug__
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,21:31:47,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer,NaN,0
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,21:31:48,ServerMain/ERROR]: Failed to load properties f...,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,0
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,21:31:48,ServerMain/WARN]: Failed to load eula.txt,ServerMain/WARN,Failed to load eula.txt,NaN,0
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,21:31:48,ServerMain/INFO]: You need to agree to the EUL...,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,0
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,21:34:03,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer,NaN,0
...,...,...,...,...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,18:50:50,User Authenticator #5/INFO]: UUID of player AR...,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,0
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,18:50:50,Server thread/INFO]: ARTEXX joined the game,Server thread/INFO,ARTEXX joined the game,NaN,0
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,18:50:50,Server thread/INFO]: ARTEXX[/[0:0:0:0:0:0:0:1]...,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,0
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,18:52:31,Server thread/INFO]: ARTEXX issued server comm...,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,0


In [ ]:
# vibo: основные логи сервера в отдельный столбец
row_logs['serv_mess'] = row_logs[row_logs['__plug__'] == 0]['row_logs_split_2']

row_logs

,row_logs,date,row_logs_split_0,time,row_logs_split_1,serv_teg,row_logs_split_2,plug_name,__plug__,serv_mess
0,2023-02-13 [21:31:47] [ServerMain/INFO]: Build...,2023-02-13,[21:31:47] [ServerMain/INFO]: Building unoptim...,21:31:47,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer,NaN,0,Building unoptimized datafixer
1,2023-02-13 [21:31:48] [ServerMain/ERROR]: Fail...,2023-02-13,[21:31:48] [ServerMain/ERROR]: Failed to load ...,21:31:48,ServerMain/ERROR]: Failed to load properties f...,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,0,Failed to load properties from file: server.p...
2,2023-02-13 [21:31:48] [ServerMain/WARN]: Faile...,2023-02-13,[21:31:48] [ServerMain/WARN]: Failed to load e...,21:31:48,ServerMain/WARN]: Failed to load eula.txt,ServerMain/WARN,Failed to load eula.txt,NaN,0,Failed to load eula.txt
3,2023-02-13 [21:31:48] [ServerMain/INFO]: You n...,2023-02-13,[21:31:48] [ServerMain/INFO]: You need to agre...,21:31:48,ServerMain/INFO]: You need to agree to the EUL...,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,0,You need to agree to the EULA in order to run...
5,2023-02-13 [21:34:03] [ServerMain/INFO]: Build...,2023-02-13,[21:34:03] [ServerMain/INFO]: Building unoptim...,21:34:03,ServerMain/INFO]: Building unoptimized datafixer,ServerMain/INFO,Building unoptimized datafixer,NaN,0,Building unoptimized datafixer
...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09 [18:50:50] [User Authenticator #5/I...,2023-03-09,[18:50:50] [User Authenticator #5/INFO]: UUID ...,18:50:50,User Authenticator #5/INFO]: UUID of player AR...,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,0,UUID of player ARTEXX is ee86a614-d39f-3a50-a...
6578,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX joined...,18:50:50,Server thread/INFO]: ARTEXX joined the game,Server thread/INFO,ARTEXX joined the game,NaN,0,ARTEXX joined the game
6579,2023-03-09 [18:50:50] [Server thread/INFO]: AR...,2023-03-09,[18:50:50] [Server thread/INFO]: ARTEXX[/[0:0:...,18:50:50,Server thread/INFO]: ARTEXX[/[0:0:0:0:0:0:0:1]...,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,0,ARTEXX[/[0:0:0:0:0:0:0:1
6580,2023-03-09 [18:52:31] [Server thread/INFO]: AR...,2023-03-09,[18:52:31] [Server thread/INFO]: ARTEXX issued...,18:52:31,Server thread/INFO]: ARTEXX issued server comm...,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,0,ARTEXX issued server command: /farwand


In [ ]:
# vibo: логи плагинов сложил в отдельный столбец
row_logs['plug_mess'] = row_logs[row_logs['__plug__'] == 1]['row_logs_split_2'].map(lambda x: x[(x.find(']')) + 2:])

row_logs[['date', 'time', 'serv_teg', 'serv_mess', 'plug_name', 'plug_mess']]

,date,time,serv_teg,serv_mess,plug_name,plug_mess
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN


In [ ]:
row_logs = row_logs[['date', 'time', 'serv_teg', 'serv_mess', 'plug_name', 'plug_mess']]
row_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN


In [ ]:
row_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN


Что у нас есть: столбцы дата и время сообщения, также есть первый блок информации, основное сообщение сервера (столбец `serv_teg`) им будем заниматься в первую очередь. Второй блок информации - дополнительные сообщения сервера (`serv_mess`), к ним перейдем позже. В конце два столбца с выделенным имененем плагина и сообщениями плагинов.

In [ ]:
# vibo: смотрим пропуски
row_logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6544 entries, 0 to 6581
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       6544 non-null   object
 1   time       6544 non-null   object
 2   serv_teg   6544 non-null   object
 3   serv_mess  6092 non-null   object
 4   plug_name  452 non-null    object
 5   plug_mess  452 non-null    object
dtypes: object(6)
memory usage: 357.9+ KB


In [ ]:
# vibo: сводная статистика
row_logs.describe()

,date,time,serv_teg,serv_mess,plug_name,plug_mess
count,6544,6544,6544,6092,452,452
unique,10,1306,34,913,3,51
top,2023-03-06,21:13:39,Server thread/INFO,Preparing spawn area: 0%,WorldEdit,
freq,1093,115,5294,674,218,44


In [ ]:
row_logs[['date', 'serv_teg']].groupby(['date'])['date'].count().to_frame(name='count_mess').reset_index()

,date,count_mess
0,2023-02-13,568
1,2023-02-14,360
2,2023-02-17,282
3,2023-03-03,594
4,2023-03-04,1018
5,2023-03-05,827
6,2023-03-06,1093
7,2023-03-07,331
8,2023-03-08,921
9,2023-03-09,550


In [ ]:
# vibo: суммарное количество сообщений
row_logs["date"].count().sum()

6544

In [ ]:
# vibo: дни наблюдения
list(row_logs['date'].unique())

['2023-02-13 ',
 '2023-02-14 ',
 '2023-02-17 ',
 '2023-03-03 ',
 '2023-03-04 ',
 '2023-03-05 ',
 '2023-03-06 ',
 '2023-03-07 ',
 '2023-03-08 ',
 '2023-03-09 ']

In [ ]:
# vibo: количество дней наблюдения
len(list(row_logs['date'].unique()))

10

In [ ]:
# vibo: сохраняем предобработанный лог (по дефолту index = True, сохраняется)
row_logs.to_csv('/content/drive/MyDrive/minecraft/logs.csv')